In [5]:
import re
import cv2
import csv
import time
import pyARIS
import threading
import subprocess
import numpy as np
import datumaro as dm
from my_utils import *
from PIL import Image
from tqdm import tqdm
from dateutil import parser
import matplotlib.pyplot as plt
from multiprocessing import Process

sonarDataDir = "./sonar/"
outputDir = "./output/"

def preprocess_data_and_generate_csv_file():
    output_video_folder_path = os.path.join(outputDir, "video")
    create_folder(output_video_folder_path)
    r = "(.+)_(.+)\.zip"

    with open(os.path.join(outputDir, "fileInfo.csv"), "w") as f:
        csvWriter = csv.writer(f)
        csvWriter.writerow(["filename", "vid_path", "anno_path"])

        for item in os.listdir(os.path.join(sonarDataDir, "annotation")):
            match = re.match(r, item)
            date = parser.parse(match[1])
            time = match[2]

            ARISfolderName = "ARIS_%s" %(date.strftime("%Y_%m_%d"))
            ARISFolderPath = os.path.join(sonarDataDir, "ARIS", ARISfolderName)
            ARISFilePath = os.path.join(ARISFolderPath, "%s_%s.aris" %(date.strftime("%Y-%m-%d"), time))
            
            # From annotation folder, find the corresponding ARIS file and preprocess
            if os.path.exists(ARISFolderPath) and os.path.exists(ARISFilePath):
                print("Processing %s" %(ARISFilePath))
                ARISFileName = os.path.splitext(os.path.basename(ARISFilePath))[0]
                videoFolderPath = os.path.join(output_video_folder_path, ARISfolderName)
                create_folder(videoFolderPath)

                ARISdata, _ = pyARIS.DataImport(ARISFilePath)
                pyARIS.VideoExportOriginal(ARISdata, filename=os.path.join(videoFolderPath, "%s.mp4" %(ARISFileName)))

                csvWriter.writerow([ARISFileName, os.path.join(videoFolderPath, "%s.mp4" %(ARISFileName)), os.path.join(sonarDataDir, "annotation", item)])

def annotation_converter_helper(datum_proj_path, coco_output_proj_path):
    dataset = dm.Dataset.import_from(datum_proj_path, 'datumaro')
    dataset.export(coco_output_proj_path, 'coco')

def convert_annotation_to_coco():
    datum_proj_path = os.path.join(outputDir, "datum_proj")
    datum_anno_path = os.path.join(outputDir, "annotation")
    coco_proj_path = os.path.join(outputDir, "coco_proj")

    subprocess.run(["python3", "datum_create_dataset.py", os.path.join(outputDir, "fileInfo.csv"), 
                    "--proj-path", datum_proj_path,
                    "--anno-path", datum_anno_path])
    pList = []
    for item in os.listdir(datum_proj_path):
        if os.path.isdir(os.path.join(datum_proj_path, item)):
            datum_input_proj_path = os.path.join(datum_proj_path, item)
            coco_output_proj_path = os.path.join(coco_proj_path, item)

            p = Process(target=annotation_converter_helper, args=(datum_input_proj_path, coco_output_proj_path))
            pList.append(p)
            p.start()
    
    for p in pList:
        p.join()
    
    print("\n\nDone!")

if __name__ == "__main__":
    preprocess_data_and_generate_csv_file()
    convert_annotation_to_coco()

<>:23: DeprecationWarning: invalid escape sequence \.
<>:23: DeprecationWarning: invalid escape sequence \.
/tmp/ipykernel_17280/3215951093.py:23: DeprecationWarning: invalid escape sequence \.
  r = "(.+)_(.+)\.zip"


Processing ./sonar/ARIS/ARIS_2020_05_31/2020-05-31_015000.aris


  0%|          | 0/2768 [00:00<?, ?it/s]ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
 

Processing ./sonar/ARIS/ARIS_2020_05_31/2020-05-31_020000.aris


  0%|          | 0/2742 [00:00<?, ?it/s]ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/tmp/build/80754af9/ffmpeg_1587154242452/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
 

Archive:  ./sonar/annotation/2020-05-31_020000.zip
 extracting: /home/henry/projects/salmon_counting/aris-file-proc/output/annotation/cvat_2020-05-31_020000/annotations.xml  


2023-08-26 17:31:15 INFO     Exporting as datumaro to /home/henry/projects/salmon_counting/aris-file-proc/output/datum_proj/2020-05-31_020000
2023-08-26 17:31:15 INFO     Generating seqinfo.ini file to /home/henry/projects/salmon_counting/aris-file-proc/output/datum_proj_inis/2020-05-31_020000/seqinfo.ini
2023-08-26 17:31:15 INFO     Unzipping and importing CVAT...
/home/henry/anaconda3/envs/tf/lib/python3.9/site-packages/datumaro/util/annotation_util.py:157: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(mask_utils.iou(a, b, [not is_bbox]))


Archive:  ./sonar/annotation/2020-05-31_015000.zip
 extracting: /home/henry/projects/salmon_counting/aris-file-proc/output/annotation/cvat_2020-05-31_015000/annotations.xml  


2023-08-26 17:31:15 INFO     Exporting as datumaro to /home/henry/projects/salmon_counting/aris-file-proc/output/datum_proj/2020-05-31_015000
2023-08-26 17:31:15 INFO     Generating seqinfo.ini file to /home/henry/projects/salmon_counting/aris-file-proc/output/datum_proj_inis/2020-05-31_015000/seqinfo.ini


Done
